# Design of LSTM and GRU RNN for classification of IMDB reviews

### Step-1

In [46]:
import pandas as pd    # to load dataset
import numpy as np     # for mathematic equation
from nltk.corpus import stopwords   # to get collection of stopwords
from sklearn.model_selection import train_test_split       # for splitting dataset
from tensorflow.keras.preprocessing.text import Tokenizer  # to encode text to int
from tensorflow.keras.preprocessing.sequence import pad_sequences   # to do padding or truncating
from tensorflow.keras.models import Sequential     # the model
from tensorflow.keras.layers import Embedding, LSTM, Dense # layers of the architecture
from tensorflow.keras.callbacks import ModelCheckpoint   # save model
from tensorflow.keras.models import load_model   # load saved model
import re
from tensorflow.keras.layers import Bidirectional

In [19]:
data = pd.read_csv('IMDB Dataset.csv')

print(data)

                                                  review sentiment
0      One of the other reviewers has mentioned that ...  positive
1      A wonderful little production. <br /><br />The...  positive
2      I thought this was a wonderful way to spend ti...  positive
3      Basically there's a family where a little boy ...  negative
4      Petter Mattei's "Love in the Time of Money" is...  positive
...                                                  ...       ...
49995  I thought this movie did a down right good job...  positive
49996  Bad plot, bad dialogue, bad acting, idiotic di...  negative
49997  I am a Catholic taught in parochial elementary...  negative
49998  I'm going to have to disagree with the previou...  negative
49999  No one expects the Star Trek movies to be high...  negative

[50000 rows x 2 columns]


### Step-2

In [24]:
english_stops = set(stopwords.words('english'))

In [25]:
def load_dataset():
    df = pd.read_csv('IMDB Dataset.csv')
    x_data = df['review']       # Reviews/Input
    y_data = df['sentiment']    # Sentiment/Output

    # PRE-PROCESS REVIEW
    x_data = x_data.replace({'<.*?>': ''}, regex = True)          # remove html tag
    x_data = x_data.replace({'[^A-Za-z]': ' '}, regex = True)     # remove non alphabet
    x_data = x_data.apply(lambda review: [w for w in review.split() if w not in english_stops])  # remove stop words
    x_data = x_data.apply(lambda review: [w.lower() for w in review])   # lower case
    
    # ENCODE SENTIMENT -> 0 & 1
    y_data = y_data.replace('positive', 1)
    y_data = y_data.replace('negative', 0)

    return x_data, y_data

x_data, y_data = load_dataset()

print('Reviews')
print(x_data, '\n')
print('Sentiment')
print(y_data)

Reviews
0        [one, reviewers, mentioned, watching, oz, epis...
1        [a, wonderful, little, production, the, filmin...
2        [i, thought, wonderful, way, spend, time, hot,...
3        [basically, family, little, boy, jake, thinks,...
4        [petter, mattei, love, time, money, visually, ...
                               ...                        
49995    [i, thought, movie, right, good, job, it, crea...
49996    [bad, plot, bad, dialogue, bad, acting, idioti...
49997    [i, catholic, taught, parochial, elementary, s...
49998    [i, going, disagree, previous, comment, side, ...
49999    [no, one, expects, star, trek, movies, high, a...
Name: review, Length: 50000, dtype: object 

Sentiment
0        1
1        1
2        1
3        0
4        1
        ..
49995    1
49996    0
49997    0
49998    0
49999    0
Name: sentiment, Length: 50000, dtype: int64


### Step-3

In [26]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size = 0.2)

print('Train Set')
print(x_train, '\n')
print(x_test, '\n')
print('Test Set')
print(y_train, '\n')
print(y_test)

Train Set
49688    [is, possible, movie, get, worse, there, bunch...
16549    [barney, friends, probably, worst, kids, show,...
4815     [minutes, nope, thing, minutes, tops, if, cann...
42233    [well, safe, say, subconscious, cruelty, one, ...
49020    [this, program, certainly, favorite, non, sitc...
                               ...                        
43842    [everything, i, remember, excellent, great, ca...
1803     [christian, propaganda, lots, fear, mongering,...
5641     [bought, movie, bargain, bin, rogers, video, s...
10911    [before, watching, film, i, low, expectations,...
25513    [in, light, recent, quite, good, batman, brave...
Name: review, Length: 40000, dtype: object 

7319     [i, understand, people, the, film, neither, go...
26626    [the, world, war, one, best, documentaries, wo...
2549     [let, face, perfect, production, hamlet, simpl...
22197    [fata, morgana, absolute, masterpiece, it, wer...
27648    [in, past, years, i, rented, bad, movies, comp...
 

In [30]:
def get_max_length():
    review_length = []
    for review in x_train:
        review_length.append(len(review))

    return int(np.ceil(np.mean(review_length)))

In [31]:
# ENCODE REVIEW
token = Tokenizer(lower=False)    # no need lower, because already lowered the data in load_data()
token.fit_on_texts(x_train)
x_train = token.texts_to_sequences(x_train)
x_test = token.texts_to_sequences(x_test)

max_length = get_max_length()

x_train = pad_sequences(x_train, maxlen=max_length, padding='post', truncating='post')
x_test = pad_sequences(x_test, maxlen=max_length, padding='post', truncating='post')

total_words = len(token.word_index) + 1   # add 1 because of 0 padding

print('Encoded X Train\n', x_train, '\n')
print('Encoded X Test\n', x_test, '\n')
print('Maximum review length: ', max_length)

Encoded X Train
 [[ 506  530    3 ...    0    0    0]
 [4001  264  144 ... 2483  458    1]
 [ 134 6077   65 ...    0    0    0]
 ...
 [1123    3 4652 ...    0    0    0]
 [1588   66    4 ...    0    0    0]
 [  49  558 1108 ...    0    0    0]] 

Encoded X Test
 [[    1   288    21 ...     0     0     0]
 [    2    85   201 ...     0     0     0]
 [  186   301   299 ... 11250  3431  7797]
 ...
 [    1   206  1927 ...     0     0     0]
 [  539     3   445 ...     0     0     0]
 [ 9250 38128  8622 ...   890  1907 38712]] 

Maximum review length:  130


### Step-4

In [33]:
# ARCHITECTURE
EMBED_DIM = 32
LSTM_OUT = 64

model = Sequential()
model.add(Embedding(total_words, EMBED_DIM, input_length = max_length))
model.add(LSTM(LSTM_OUT))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 130, 32)           2956128   
                                                                 
 lstm_1 (LSTM)               (None, 64)                24832     
                                                                 
 dense_1 (Dense)             (None, 64)                4160      
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                                 
Total params: 2,985,185
Trainable params: 2,985,185
Non-trainable params: 0
_________________________________________________________________
None


In [34]:
model.fit(x_train, y_train, batch_size = 128, epochs = 10)

Epoch 1/10
313/313 [==============================] - 33s 98ms/step - loss: 0.6779 - accuracy: 0.5434
Epoch 2/10
313/313 [==============================] - 31s 98ms/step - loss: 0.4839 - accuracy: 0.7456
Epoch 3/10
313/313 [==============================] - 30s 97ms/step - loss: 0.2134 - accuracy: 0.9212
Epoch 4/10
313/313 [==============================] - 30s 97ms/step - loss: 0.1139 - accuracy: 0.9644
Epoch 5/10
313/313 [==============================] - 31s 98ms/step - loss: 0.0677 - accuracy: 0.9804
Epoch 6/10
313/313 [==============================] - 31s 98ms/step - loss: 0.0454 - accuracy: 0.9880
Epoch 7/10
313/313 [==============================] - 31s 98ms/step - loss: 0.0360 - accuracy: 0.9894
Epoch 8/10
313/313 [==============================] - 31s 98ms/step - loss: 0.0326 - accuracy: 0.9921
Epoch 9/10
313/313 [==============================] - 31s 99ms/step - loss: 0.0292 - accuracy: 0.9921
Epoch 10/10
313/313 [==============================] - 31s 98ms/step - loss: 0.026

In [41]:
model.evaluate(x_test,y_test)

313/313 [==============================] - 4s 13ms/step - loss: 0.5409 - accuracy: 0.8558


[0.5409194827079773, 0.8557999730110168]

### Step-5

In [43]:
# ARCHITECTURE
EMBED_DIM = 32

model1 = Sequential()
model1.add(Embedding(total_words, EMBED_DIM, input_length = max_length))
model1.add(LSTM(32))
model1.add(Dense(64, activation='relu'))
model1.add(Dense(64, activation='relu'))
model1.add(Dense(1, activation='sigmoid'))
model1.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

print(model1.summary())

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_6 (Embedding)     (None, 130, 32)           2956128   
                                                                 
 lstm_9 (LSTM)               (None, 32)                8320      
                                                                 
 dense_5 (Dense)             (None, 64)                2112      
                                                                 
 dense_6 (Dense)             (None, 64)                4160      
                                                                 
 dense_7 (Dense)             (None, 1)                 65        
                                                                 
Total params: 2,970,785
Trainable params: 2,970,785
Non-trainable params: 0
_________________________________________________________________
None


In [44]:
model1.fit(x_train, y_train, batch_size = 128, epochs = 5)

Epoch 1/5
313/313 [==============================] - 21s 61ms/step - loss: 0.4573 - accuracy: 0.7458
Epoch 2/5
313/313 [==============================] - 19s 61ms/step - loss: 0.1963 - accuracy: 0.9271
Epoch 3/5
313/313 [==============================] - 19s 61ms/step - loss: 0.1056 - accuracy: 0.9646
Epoch 4/5
313/313 [==============================] - 19s 62ms/step - loss: 0.0621 - accuracy: 0.9801
Epoch 5/5
313/313 [==============================] - 19s 61ms/step - loss: 0.0432 - accuracy: 0.9863


### Step-6

In [48]:
# ARCHITECTURE
EMBED_DIM = 32
LSTM_OUT = 64

model2 = Sequential()
model2.add(Embedding(total_words, EMBED_DIM, input_length = max_length))
model2.add(Bidirectional(LSTM(LSTM_OUT)))
model2.add(Dense(64, activation='relu'))
model2.add(Dense(1, activation='sigmoid'))
model2.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

print(model2.summary())

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_9 (Embedding)     (None, 130, 32)           2956128   
                                                                 
 bidirectional_1 (Bidirectio  (None, 128)              49664     
 nal)                                                            
                                                                 
 dense_10 (Dense)            (None, 64)                8256      
                                                                 
 dense_11 (Dense)            (None, 1)                 65        
                                                                 
Total params: 3,014,113
Trainable params: 3,014,113
Non-trainable params: 0
_________________________________________________________________
None


In [52]:
model2.fit(x_train, y_train, batch_size = 128)

313/313 [==============================] - 51s 164ms/step - loss: 0.1823 - accuracy: 0.9330
